In [2]:
# load in relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate, GridSearchCV

In [8]:
# load in our dataset
cancer = pd.read_csv('../../01_materials/notebooks/dataset/wdbc.csv')
cancer

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.70,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0.7655,2.4630,5.203,99.04,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0.4564,1.0750,3.425,48.55,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0.7260,1.5950,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [ ]:
cancer['diagnosis'] = cancer ['diagnosis'].replace({"M" : "Malignant", "B": "Benign"}) #like excel replace
cancer 

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,Malignant,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,Malignant,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,Malignant,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,Malignant,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,Malignant,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,Malignant,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.70,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,Malignant,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0.7655,2.4630,5.203,99.04,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,Malignant,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0.4564,1.0750,3.425,48.55,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,Malignant,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0.7260,1.5950,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [ ]:
# copy our original data to avoid changing original data
standardized_caner = cancer.copy()

columns_to_exclude = ['id', 'diagnosis']
columns_to_scale = standardized_caner.columns.difference(columns_to_exclude) #.columns.difference
columns_to_scale

Index(['area_mean', 'area_se', 'area_worst', 'compactness_mean',
       'compactness_se', 'compactness_worst', 'concave points_mean',
       'concave points_se', 'concave points_worst', 'concavity_mean',
       'concavity_se', 'concavity_worst', 'fractal_dimension_mean',
       'fractal_dimension_se', 'fractal_dimension_worst', 'perimeter_mean',
       'perimeter_se', 'perimeter_worst', 'radius_mean', 'radius_se',
       'radius_worst', 'smoothness_mean', 'smoothness_se', 'smoothness_worst',
       'symmetry_mean', 'symmetry_se', 'symmetry_worst', 'texture_mean',
       'texture_se', 'texture_worst'],
      dtype='object')

In [ ]:
#fit to have a standard deviation of 1
scaler = StandardScaler()   #initialize our scaler
standardized_caner[columns_to_scale] = scaler.fit_transform(standardized_caner[columns_to_scale]) #why can we call "standardized_caner" again?
#fit - learn the mean and SD of the column, then transform it to mean of 0 and SD of 1 --> value subtracted by mean and then divide by SD
standardized_caner


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,Malignant,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,2.217515,2.255747,2.489734,-0.565265,2.833031,2.487578,-0.214002,1.316862,0.724026,0.660820,1.148757,0.907083,1.886690,-1.359293,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015
1,842517,Malignant,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,0.001392,-0.868652,0.499255,-0.876244,0.263327,0.742402,-0.605351,-0.692926,-0.440780,0.260162,-0.805450,-0.099444,1.805927,-0.369203,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190
2,84300903,Malignant,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,0.939685,-0.398008,1.228676,-0.780083,0.850928,1.181336,-0.297005,0.814974,0.213076,1.424827,0.237036,0.293559,1.511870,-0.023974,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391
3,84348301,Malignant,-0.768909,0.253732,-0.592687,-0.764464,3.283553,3.402909,1.915897,1.451707,2.867383,4.910919,0.326373,-0.110409,0.286593,-0.288378,0.689702,2.744280,0.819518,1.115007,4.732680,2.047511,-0.281464,0.133984,-0.249939,-0.550021,3.394275,3.893397,1.989588,2.175786,6.046041,4.935010
4,84358402,Malignant,1.750297,-1.151816,1.776573,1.826229,0.280372,0.539340,1.371011,1.428493,-0.009560,-0.562450,1.270543,-0.790244,1.273189,1.190357,1.483067,-0.048520,0.828471,1.144205,-0.361092,0.499328,1.298575,-1.466770,1.338539,1.220724,0.220556,-0.313395,0.613179,0.729259,-0.868353,-0.397100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,Malignant,2.110995,0.721473,2.060786,2.343856,1.041842,0.219060,1.947285,2.320965,-0.312589,-0.931027,2.782080,0.071025,2.379583,2.604187,1.086384,0.191805,0.666001,2.067178,-1.138416,0.167980,1.901185,0.117700,1.752563,2.015301,0.378365,-0.273318,0.664512,1.629151,-1.360158,-0.709091
565,926682,Malignant,1.704854,2.085134,1.615931,1.723842,0.102458,-0.017833,0.693043,1.263669,-0.217664,-1.058611,1.300499,2.260938,1.156857,1.291565,-0.424010,-0.069758,0.252202,0.808431,-0.189161,-0.490556,1.536720,2.047399,1.421940,1.494959,-0.691230,-0.394820,0.236573,0.733827,-0.531855,-0.973978
566,926954,Malignant,0.702284,2.045574,0.672676,0.577953,-0.840484,-0.038680,0.046588,0.105777,-0.809117,-0.895587,0.184892,-0.257371,0.276693,0.180698,-0.379342,0.661277,0.510827,0.612157,-0.891416,0.036727,0.561361,1.374854,0.579001,0.427906,-0.809587,0.350735,0.326767,0.414069,-1.104549,-0.318409
567,927241,Malignant,1.838341,2.336457,1.982524,1.735218,1.525767,3.272144,3.296944,2.658866,2.137194,1.043695,1.157935,0.686088,1.438530,1.009503,-0.173000,2.017716,1.302285,0.785721,0.326634,0.904057,1.961239,2.237926,2.303601,1.653171,1.430427,3.904848,3.197605,2.289985,1.919083,2.219635


In [ ]:
#usually we set 75% of data for training and 25% for testing
#set a seed for reproducibility
np.random.seed(1) #can be any number instead of 1

#split the data
cancer_train, cancer_test = train_test_split(standardized_caner, train_size=0.75, shuffle=True, stratify=standardized_caner['diagnosis']) 
#shuffle the data so that the order of data is not affecting the first 75% of the data - by default it is TRUE
#stratify = diagnosis -- tell the computer that we want to base on this parameter to train
#we can have 2 variables before = as this will output 2 variables - where the first variable = train_size
cancer_train

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
357,901028,Benign,-0.073075,-0.716655,-0.142066,-0.174028,-0.635527,-0.936601,-0.926297,-0.723241,-1.415174,-0.562450,-0.544529,0.265160,-0.558919,-0.431169,-0.467679,-0.980254,-0.883294,-0.933377,-0.617779,-0.646017,-0.240048,-0.015832,-0.313383,-0.327294,-0.748217,-0.976252,-1.052282,-0.899073,-0.871589,-0.710199
361,901041,Benign,-0.234963,0.530653,-0.277170,-0.309407,-0.750104,-0.769638,-0.695035,-0.636573,0.012345,-0.827542,-0.516377,0.584485,-0.414865,-0.425889,-0.514347,-0.281020,-0.462332,-0.875468,-0.272705,-0.695568,-0.428493,0.573662,-0.426569,-0.455973,-0.805204,-0.557036,-0.724371,-0.890242,-0.426699,-0.962341
212,8810703,Malignant,3.971288,-0.190738,3.976130,5.244841,1.269571,0.895628,2.903973,2.852321,-0.597362,-1.069952,8.906909,0.470181,9.461986,10.676614,2.136427,0.125297,1.060902,0.368843,3.303954,0.257626,2.452025,-1.173652,2.419765,2.845036,-0.796437,-0.653093,0.229857,0.683579,-2.026684,-1.590202
527,91813702,Benign,-0.507616,-1.633519,-0.536668,-0.530110,-0.450497,-0.782146,-0.743497,-0.579053,-0.447674,-0.668770,-1.041517,-1.308420,-1.036920,-0.690459,-1.120038,-0.911510,-0.767974,-0.982364,-1.120254,-0.918736,-0.550672,-1.043377,-0.596944,-0.554943,-0.138898,-0.298127,-0.446594,-0.115817,0.338511,-0.444757
21,8510824,Benign,-1.313080,-1.593959,-1.302806,-1.083572,0.429819,-0.747086,-0.743748,-0.726337,0.012345,0.886341,-0.461517,-0.435539,-0.473774,-0.542058,0.855042,-0.623623,-0.399334,0.391552,-0.032970,-0.312777,-1.250611,-1.631243,-1.254913,-0.994422,0.001377,-0.887193,-0.880434,-0.796903,-0.729224,-0.344455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,9010877,Benign,-0.206561,-0.544452,-0.267285,-0.291490,-1.209121,-0.897940,-0.841049,-0.881874,-0.590060,-0.820454,-0.890652,-1.096686,-0.905935,-0.596622,-0.882362,-0.725342,-0.576392,-1.023890,-0.924107,-0.650934,-0.318739,-0.647666,-0.402145,-0.381613,-0.485202,-0.551311,-0.651448,-0.681180,-0.258450,-0.450299
434,908469,Benign,0.208100,-0.546779,0.120315,0.053500,-0.506718,-0.636788,-0.694784,-0.519727,-0.871183,-0.817619,-0.369482,-0.995265,-0.620798,-0.361204,-0.428344,-0.493401,-0.573077,-0.570189,-0.609303,-0.615379,0.008451,-0.836565,-0.147774,-0.181211,-0.463284,-0.631464,-0.720533,-0.531351,-0.607890,-0.868688
299,892399,Benign,-1.027362,0.884367,-1.034658,-0.912073,0.365770,-0.689284,-0.801626,-0.778182,-0.425768,0.391598,-0.427230,-0.133996,-0.285662,-0.435130,1.043049,-0.617475,-0.440448,0.114173,1.706929,-0.689894,-1.105653,-0.237300,-1.106878,-0.910393,-0.792053,-1.069510,-1.106350,-1.269232,-1.089989,-0.896396
488,913512,Benign,-0.695066,-0.725963,-0.678775,-0.666627,1.169940,-0.221940,-0.577646,-0.453952,0.151081,0.171869,-0.122251,-0.114038,-0.154479,-0.280898,0.652367,-0.701869,-0.443764,-0.020461,0.118379,-0.220106,-0.610726,-0.665579,-0.616305,-0.581488,0.886862,-0.677903,-0.591000,-0.250572,-0.156530,-0.205361


In [ ]:
cancer_train.info() #426 data -- 75%

<class 'pandas.core.frame.DataFrame'>
Index: 426 entries, 164 to 284
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       426 non-null    int64  
 1   diagnosis                426 non-null    object 
 2   radius_mean              426 non-null    float64
 3   texture_mean             426 non-null    float64
 4   perimeter_mean           426 non-null    float64
 5   area_mean                426 non-null    float64
 6   smoothness_mean          426 non-null    float64
 7   compactness_mean         426 non-null    float64
 8   concavity_mean           426 non-null    float64
 9   concave points_mean      426 non-null    float64
 10  symmetry_mean            426 non-null    float64
 11  fractal_dimension_mean   426 non-null    float64
 12  radius_se                426 non-null    float64
 13  texture_se               426 non-null    float64
 14  perimeter_se             426 

In [ ]:
#step 1. initialize the model
knn = KNeighborsClassifier(n_neighbors=5)

#step 2 define x and y (response)
X = cancer_train[["perimeter_mean","concavity_mean"]]
y = cancer_train['diagnosis'] #single bracket only!

#step 3. foit our model to our data
knn.fit(X,y)

c:\Users\Chan Yat Yan\Desktop\DSI_2025\LCR\lcr-env\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [ ]:
#step 4 predict on the test set
cancer_test["predicted"] = knn.predict(cancer_test[["perimeter_mean","concavity_mean"]]) #creating the column "Predicted" at the same time
cancer_test[["diagnosis", "predicted"]] #just showing the column "predicted" and "diagnosis"
#dang, we need to use [["column A", "column B"]] but not ([])

,diagnosis,predicted
357,Benign,Benign
361,Benign,Benign
212,Malignant,Malignant
527,Benign,Benign
21,Benign,Benign
...,...,...
364,Benign,Benign
434,Benign,Benign
299,Benign,Benign
488,Benign,Benign


In [42]:
#to test the accuracy
knn.score(cancer_test[["perimeter_mean","concavity_mean"]], cancer_test['diagnosis'])

0.9230769230769231

In [ ]:
#confusion matrix

pd.crosstab(cancer_test['diagnosis'],cancer_test['predicted']) #this one output the result directly

predicted,Benign,Malignant
diagnosis,,
Benign,88,2
Malignant,9,44


In [ ]:
#precision = when the model predicted malignant, how often is it actually correct (44/46, from the above) - It cares about false positives
precision_score(y_true = cancer_test['diagnosis'], y_pred=cancer_test['predicted'], pos_label="Malignant")

0.9565217391304348

In [47]:
#the precision can also be on Benign if we define it with benign (88/97)
precision_score(y_true = cancer_test['diagnosis'], y_pred=cancer_test['predicted'], pos_label="Benign")

0.9072164948453608

In [ ]:
#recall = of all the actual malianant tumors, how many did the model successfuly identify? It cares about false negatives
recall_score(y_true=cancer_test['diagnosis'], y_pred= cancer_test['predicted'], pos_label="Malignant")

0.8301886792452831

In [ ]:
#split into subtrain and validation - to test the best k
cancer_subtrain, cancer_validation = train_test_split(cancer_train, train_size=0.75, stratify=cancer_train["diagnosis"])

In [51]:
#fit our model to the subtraining data

knn = KNeighborsClassifier(n_neighbors=3)
X = cancer_subtrain[["perimeter_mean","concavity_mean"]]
y = cancer_subtrain["diagnosis"]
knn.fit(X,y)

,n_neighbors,3
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [ ]:
acc = knn.score(cancer_validation[["perimeter_mean","concavity_mean"]], cancer_validation["diagnosis"]) #set a variable to test for accuracy
acc

0.8785046728971962

In [ ]:
#to avoid good split or bad split (train_test_split), it's better to use cross-validation
#running the splitting module again will give different values as the data is shuffled
knn = KNeighborsClassifier(n_neighbors=3)
X = cancer_train[["perimeter_mean","concavity_mean"]]
y = cancer_train["diagnosis"]

returned_dictionary = cross_validate(estimator = knn, cv = 5, X = X, y=y) # mind the capitalization!!!
#cv = 5 -- split the data into 5 parts


In [56]:
cv_5_df = pd.DataFrame(returned_dictionary)
cv_5_df

,fit_time,score_time,test_score
0,0.005024,0.002992,0.930233
1,0.001989,0.003979,0.894118
2,0.002003,0.002962,0.870588
3,0.002996,0.003987,0.952941
4,0.001015,0.003002,0.917647
